In [ ]:
import pandas as pd 
import pymongo 
from jsonpath import jsonpath
import sys 
from typing import List
from pydantic import BaseModel, Field

db_client = pymongo.MongoClient("192.168.8.10", 27017)
db_names = db_client.list_database_names()
print(db_names)
shipment_collection = db_client['carrier']["shipments"]
ship = shipment_collection.find_one({"_id": "028-7008415-7713125"}, )

class PickSlipVO(BaseModel):
    """
    The response model for the getPickSlip endpoint
    """
    date: str = Field(description="The date of the pick slip")
    carrier: str = Field(description="The name of the carrier")
    orderId: str = Field(description="The id of the b2c order")
    trackId: str = Field(description="The id of the tracking number")
    parcelNumber: str = Field(description="The parcel number of the shipment")
    sku: str = Field(description="The sku of the product")
    title: str = Field(description="The title of the product")
    quantity: int = Field(description="The quantity of the product")
    storageLocation: str = Field(description="The storage location of the product")
    imageUrl: str = Field(description="The image url of the product")
    street1: str = Field(description="The street address of the recipient")



In [ ]:
ids = ["028-7008415-7713125", "304-1808739-3152312"]
data = shipment_collection.find({"_id": {"$in": ids}})
data = list(data)

In [ ]:
import requests
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
url = "http://127.0.0.1:5018/api/v1/carriers/gls/shipments/pick?references=304-1808739-3152312%3B306-2843703-1033918"
response = requests.get(url).json()
pickitem = response['data']
print(pickitem)

In [ ]:
df = pd.DataFrame(pickitem)
# df.to_excel('pick-slip.xlsx')
df

In [ ]:
import io
df = pd.DataFrame(pickitem)
# 创建一个新的工作簿和工作表
wb = Workbook()
ws = wb.active
# 设置单元格宽度
ws.column_dimensions['A'].width = 21
ws.column_dimensions['B'].width = 15
ws.column_dimensions['C'].width = 15
ws.column_dimensions['D'].width = 4
ws.column_dimensions['E'].width = 15
ws.column_dimensions['F'].width = 10
ws.column_dimensions['G'].width = 8
# 设置标题行
headers = ["OrderID", "Sku", "StorageLocation", "Qty", "Street",  "TrackID",  "Carrier"]
ws.append(headers)
# 按 orderId 分组
grouped = df.groupby('orderId')

for order_id, group in grouped:
    # 提取订单的基本信息
    date = group['date'].iloc[0]
    carrier = group['carrier'].iloc[0]
    track_id = group['trackId'].iloc[0]
    street = group['street1'].iloc[0]
    # 提取订单的商品信息
    items = group[['sku', 'storageLocation', 'quantity']]
    
    # 将订单基本信息添加到表格中
    row = [order_id, carrier, track_id, street]
    # ws.append(row)
    
    
    # 添加商品信息
    start_row = ws.max_row
    for i, r in enumerate(dataframe_to_rows(items, index=False, header=False)):
        sku, storage_location, quantity = r
        if i == 0:
            ws.append([order_id, sku, storage_location, quantity, street, track_id, carrier])
        else:
            ws.append(["", sku, storage_location, quantity, "", "", ""])
    
# 设置对齐方式
for row in ws.iter_rows():
    for cell in row:
        cell.alignment = Alignment(vertical='top')

# 统计每个 SKU 的数量
sku_info = df.groupby('sku').agg({'quantity': 'sum', 'storageLocation': 'first'})
# 按 SKU 排序
sorted_sku_info = sku_info.sort_values(by='quantity', ascending=False)
# 写入 SKU 数量信息到表格中
ws.append([""])
ws.append(["Date", date])
ws.append(["---------"] * 5)
ws.append(["SKU", "Storage Location", "Count"])
for sku, info in sorted_sku_info.iterrows():
    ws.append([sku, info['storageLocation'], info['quantity']])
# 保存工作簿
output = io.BytesIO()
wb.save(output)
output.seek(0)
wb.save('picking_slip.xlsx')